## NBA Hackathon: Basketball Analytics

In [1]:
__author__ = "jzqb"
__copyright__ = "None"
__credits__ = ["jzqb", "vk-allen", "kikiisme"]
__license__ = "GPL"
__version__ = "1.0.0"
__maintainer__ = "jzqb"
__email__ = "gaurav.singh@columbia.edu"
__status__ = "Prototype"

In [2]:
## import libraries
import pandas as pd
import numpy as np
from __future__ import print_function
%matplotlib inline

In [3]:
## read in the data
event_codes = pd.read_table("NBA Hackathon - Event Codes.txt")
game_lineup = pd.read_table("NBA Hackathon - Game Lineup Data Sample (50 Games).txt")
play_by_play = pd.read_table("NBA Hackathon - Play by Play Data Sample (50 Games).txt")

In [4]:
event_codes.head()

,Event_Msg_Type,Action_Type,Event_Msg_Type_Description,Action_Type_Description
0,1,0,Made Shot,No Shot
1,1,1,Made Shot,Jump Shot
2,1,3,Made Shot,Hook Shot
3,1,4,Made Shot,Tip Shot
4,1,5,Made Shot,Layup Shot


In [5]:
print("Event Codes Columns:")
print(event_codes.columns)
print(event_codes.shape)
print("---------------------")
print("Game Line Up Columns:")
print(game_lineup.columns)
print(game_lineup.shape)
print("---------------------")
print("Play by Play Columns:")
print(play_by_play.columns)
print(play_by_play.shape)

Event Codes Columns:
Index([u'Event_Msg_Type', u'Action_Type', u'Event_Msg_Type_Description',
       u'Action_Type_Description'],
      dtype='object')
(274, 4)
---------------------
Game Line Up Columns:
Index([u'Game_id', u'Period', u'Person_id', u'Team_id', u'status'], dtype='object')
(2040, 5)
---------------------
Play by Play Columns:
Index([u'Game_id', u'Event_Num', u'Event_Msg_Type', u'Period', u'WC_Time',
       u'PC_Time', u'Action_Type', u'Option1', u'Option2', u'Option3',
       u'Team_id', u'Person1', u'Person2', u'Team_id_type'],
      dtype='object')
(22808, 14)


In [6]:
flat_table = pd.merge(play_by_play, event_codes, left_on = ["Event_Msg_Type", "Action_Type"], 
                               right_on = ["Event_Msg_Type", "Action_Type"], how = "inner")

In [7]:
## lost one row on the merge
flat_table.shape

(22807, 16)

In [8]:
flat_table = flat_table.sort_values(["Game_id", "Period", "PC_Time", "WC_Time", "Event_Num"], 
                         ascending= [True, True, False, True, True])

In [9]:
all_games = flat_table.Game_id.unique()

In [52]:
master = []
for game in all_games:
    for quarter in np.linspace(1, 5, 5):
        teams = game_lineup[(game_lineup['Game_id'] == game)]['Team_id'].unique()
        team_one = teams[0]
        team_two = teams[1]
        quarter_line_up = game_lineup[(game_lineup['Game_id'] == game) & (game_lineup['Period'] == quarter)]
        game_quarter_events = flat_table[(flat_table['Game_id'] == game) & (flat_table['Period'] == quarter)]
        game_quarter_events = game_quarter_events.sort_values(["Game_id", "Period", "PC_Time", "WC_Time", "Event_Num"], 
                             ascending= [True, True, False, True, True])
        team_one_active_players = list(quarter_line_up[quarter_line_up['Team_id'] == team_one]['Person_id'].unique())
        team_two_active_players = list(quarter_line_up[quarter_line_up['Team_id'] == team_two]['Person_id'].unique())

        
        for idx, row in game_quarter_events.iterrows():
            output = []
            scoring_team = row['Team_id']
            points = row['Option1']
            event_num = row['Event_Num']
            sub_flag = row['Event_Msg_Type']
            if sub_flag == 8:
                leaving_player = row['Person1']
                joining_player = row['Person2']
                quarter_line_up['Person_id'] = quarter_line_up['Person_id'].replace(leaving_player, joining_player)
                team_one_active_players = list(quarter_line_up[quarter_line_up['Team_id'] == team_one]['Person_id'].unique())
                team_two_active_players = list(quarter_line_up[quarter_line_up['Team_id'] == team_two]['Person_id'].unique())
            if scoring_team == team_one:
                team_flag = 1
                for player in team_one_active_players:
                    output.append([game, quarter, event_num, team_one, player, points])
                for player in team_two_active_players:
                    output.append([game, quarter, event_num, team_one, player, -points])
            elif scoring_team == team_two:
                team_flag = 2
                for player in team_one_active_players:
                    output.append([game, quarter, event_num, team_two, player, -points])
                for player in team_two_active_players:
                    output.append([game, quarter, event_num, team_two, player, points])
            else:
                continue
            master = master + output

/Users/gauravsingh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [63]:
master_df = pd.DataFrame(master, columns=['Game_ID', 'Period', 'Event_Num', 'Team_id', 'Player_ID', 'Player_Plus/Minus'])

In [65]:
final_out = master_df.groupby(['Game_ID', 'Player_ID'])['Player_Plus/Minus'].agg('sum').reset_index()

In [66]:
final_out.shape

(1077, 3)

In [67]:
final_out.to_csv('submission.csv', index = False)